In [9]:
# !pip install pytorch-lightning

In [40]:
import torch
import pytorch_lightning as p
from typing import Tuple
import torchmetrics

In [64]:
class Model(p.LightningModule):
    def __init__(self):
        super().__init__()
#       metrics
        self.t_exv=torchmetrics.ExplainedVariance()
        self.v_exv=torchmetrics.ExplainedVariance()
        self.model=torch.nn.Linear(1,1)
        
    def forward(self,x):
        return self.model(x)
    
    def training_step(self,batch:Tuple[torch.Tensor,torch.Tensor],batch_idx:int) -> torch.Tensor:
        xs,ys=batch
        outs=self(xs)
        loss=torch.nn.functional.mse_loss(outs,ys)
        self.t_exv(outs,ys)
        self.log('t_exv',self.t_exv,prog_bar=True)
        self.log("loss",loss)  # for saving the loss for accessing it later
        return loss
    
    def validation_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
        x,y=batch
        y_out=self(x)
        loss = torch.nn.functional.mse_loss(y_out, y) 
        self.v_exv(y_out,y)
        self.log('v_exv',self.v_exv,prog_bar=True)
        self.log("val_loss", loss)
    
    def configure_optimizers(self)-> torch.optim.Optimizer:
        optimizer=torch.optim.Adam(self.parameters(),lr=3e-4)
        return optimizer

In [65]:
class LoggingCallback(p.Callback):
    def on_train_epoch_end(self,trainer:p.Trainer,pl_module: p.LightningModule):
        epoch = trainer.current_epoch
        logs=trainer.callback_metrics

        loss = logs.get('loss')
        exv = logs.get('t_exv')
        # Perform logging or any other operations

        print(f"Epoch {epoch} - Training Loss: {loss} - EXV: {exv}")
    
    def on_validation_epoch_end(self, trainer: p.Trainer, pl_module: p.LightningModule) -> None:
      
        epoch = trainer.current_epoch
        logs=trainer.callback_metrics

        exv=logs.get('v_exv')
        loss = logs.get('val_loss')

        print(f"Epoch {epoch} - V Loss: {loss}  - EXV: {exv}")
        
    

In [66]:

class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        self.x_train = torch.linspace(0, 1, 100)
        self.y_train = self.x_train * 2 + 1
        self.x_val = torch.linspace(0, 1, 20)
        self.y_val = self.x_val * 2 + 1

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(self.x_train, self.y_train)
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
             torch.utils.data.TensorDataset(self.x_val, self.y_val)
        )


In [67]:


model = Model()
data_module = MyDataModule()

#     checkpoint_callback = ModelCheckpoint(monitor='val_accuracy', save_top_k=1, mode='max')
#     early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

trainer = p.Trainer(
        callbacks=[LoggingCallback()],
        max_epochs=20
    )

trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params
--------------------------------------------
0 | t_exv | ExplainedVariance | 0     
1 | v_exv | ExplainedVariance | 0     
2 | model | Linear            | 2     
--------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Epoch 0 - V Loss: 1.970310926437378  - EXV: -0.1868652105331421


C:\Users\ASUS\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ASUS\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Epoch 18 - V Loss: 3.1175215244293213  - EXV: 0.2546519637107849
Epoch 18 - Training Loss: 6.503756046295166 - EXV: 1.0


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19 - V Loss: 2.992220640182495  - EXV: 0.27266818284988403
Epoch 19 - Training Loss: 6.270915985107422 - EXV: 1.0


In [70]:
# trainer.test()

In [68]:
# add noram dataset
# add test